In [ ]:
import tweepy
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
def scrape_twitter(bearer_token, query="(ريحة OR odeur) (Tunis OR Sousse) -is:retweet", max_results=10):
    client = tweepy.Client(bearer_token=bearer_token)
    try:
        response = client.search_recent_tweets(query=query, max_results=max_results)
        return pd.DataFrame([{'Text': t.text, 'Date': t.created_at} for t in response.data or []])
    except Exception as e:
        print(f"Twitter error: {e}")
        return pd.DataFrame()


In [ ]:
def scrape_tripadvisor(url="https://www.tripadvisor.com/Restaurant_Review-g295401-d1425864-Reviews-Restaurant_du_Peuple-Sousse.html"):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    reviews = []
    for review in soup.find_all('div', {'data-automation': 'reviewCard'}, limit=10):
        text = review.find('div', {'data-test-target': 'review-body'}).get_text(strip=True)
        if 'smell' in text.lower() or 'tea' in text.lower():
            reviews.append({'Text': text, 'Location': 'Sousse'})
        time.sleep(5)
    return pd.DataFrame(reviews)